<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-\amily:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Regularization
              
</p>
</div>

Data Science Cohort Live NYC June 2022
<p>Phase 3: Topic 24</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>

# Objectives
- Explain the notion of "validation data"
- Use the algorithm of cross-validation (with `sklearn`)
- Explain the concept of regularization
- Use Lasso and Ridge regularization in model design

The balance between underfitting vs. overfitting:
- A central problem in machine learning

Adding more and more features to get lower error on train set.
- E.g. more polynomial columns via Polynomial transformer, etc.

#### Model has more flexibility with more fitting parameters:
- More freedom to adjust weights to lower cost function on training data: mean squared error, etc.


But freedom can be a curse.

<img src = "Images/overfitting.png" width = 300/>

- Flexibility for weights to be optimized to fit to train data exactly.


- Flexibile model can tune floppy weights unrealistically high (or low) to fit data.

- Fits data with weight vector $\textbf{w}$: 
    - many of the features may have significant coefficients 
    - large positive weights for some
    - large negative weights for others
    


Has learned a specific complex model.

But remember:
- Training data is a **sample**.

If we take another sample from population, is it likely to learn the same weight vector $\textbf{w}$?

<img src = "Images/overfitting.png" width = 300/>

Likely to learn another $\textbf{w}$ that is significantly different.

I.e. trains and learns a different specific complex model.

Keep playing this game:
- large spread in $\textbf{w}$ on different training samples.
- Components of $\textbf{w}$ floppy. Can take on large and varied values. 

Variance in "bias-variance" has a precise statistical meaning:

$$ \mathrm{Var}[\textbf{w}] $$ where  the weight vector $ \textbf{w} $ is a random variable.

An overfitted model with more features than necessary (i.e. an overflexible model):
- Typically means high $\mathrm{Var}[\textbf{w}]$.

High $\mathrm{Var}[\textbf{w}]$:
- Significantly different and specific values of $\textbf{w}$ and learned function on each training set.

Then expect that:
-  learned function doesn't represent the average tendency of target vs features very well.
- higher error on unseen/test data.
- poorer predictions.

#### Multicollinearity

We talked about this way back. But how does it increase Var[$\textbf{w}$]?

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

WHO_data = pd.read_csv("data/WHO_life.csv")
X_WHO = WHO_data.drop(columns = ["Life expectancy "])
y = WHO_data["Life expectancy "]

In [ ]:
X_WHO.head()

Many features from WHO dataset:

Regressing to find weights life expectancy

In [ ]:
X_WHO.columns

But let's take a look at a few of these and their correlations:

In [ ]:
col_selector = ['Income composition of resources', 'Schooling','Alcohol', ' thinness  1-19 years']
subsetX = X_WHO[col_selector]
sns.heatmap(subsetX.corr())
plt.show()

Let's focus on Schooling and income composite resources (ICR):

$$ Life= w_1*Alcohol + w_2*Polio + w_3*Schooling + w_4*Measles + w_5*ICR + ... $$

Correlation is very high!

In [ ]:
col_selector = ['Income composition of resources', 'Schooling']
X_WHO[col_selector].corr()

Our regression: 
- Y = life expectancy

$$ Y - \sum_{i \neq 3,5} w_i x_i = w_3 Schooling + w_5 ICR $$

- Schooling and ICR highly related:

- Implies that $w_3$ and $w_5$ introduce too much flexibility.
- Maybe could fit almost as well with just $w_3$.

- $w_3$ and $w_5$ are floppy and can become big in either direction to fit data.
- Var[$\textbf{w}$] from $w_3$ and $w_5$ high.

#### How to assess model variance: cross-validation

Could get many different training sets:
- Train weights $\textbf{w}$ for each.
- Get variance of $\textbf{w}$ 

Semi-equivalently:
- Test performance of each model on test set.
- Evaluate model performance/variance by looking at average/standard deviation of performance on test set.

Problem: 
- likely don't have this much data available to make enough independent training sets large enough to for each model to train on effectively.

#### Solution: Cross validation

<img src = "Images/crossval.png"  width = 800/>
<center> Splitting up training set </center>

Split up training set into folds:
- Training fold
- Validation fold

- For each iteration:
    - train a model.
    - Test on validation fold. 

Effectively sampling multiple training sets:
- testing each model performance on different **validation set**.
- Good for estimating model performance on average
- Good for estimating model variance as well.


So in the end:
- Performance metrics measured on validation
- We get average performance metric across all the models for each cross validation iteration.
- Get variance of performance metric.

Note: **validation set** is part of training set:
- Not part of true test/hold-out set.

Idea is that we tune / figure out how to limit variance using training set:
- Since limited data:
    - Use train set to train and estimate model performance and variance while tuning.
- But model hase "seen" validation set:

True and final evaluation:
- Measure performance on tuned model on the test set that has never been seen before.

Roughly:
- Training data is for building the model;
- Validation data is for *tweaking* the model;
- Testing data is for evaluating the model on unseen data.

- Think of **training** data as what you study for a test
- Think of **validation** data is using a practice test (note sometimes called **dev**)
- Think of **testing** data as what you use to judge the model
    - A **holdout** set is when your test dataset is never used for training (unlike in cross-validation)

![](https://scikit-learn.org/stable/_images/grid_search_workflow.png)
> Image from Scikit-Learn https://scikit-learn.org/stable/modules/cross_validation.html

1. Split data into training data and a holdout test
2. Design a model
3. Evaluate how well it generalizes with **cross-validation** (only training data)
4. Determine if we should adjust model, use cross-validation to evaluate, and repeat
5. After iteratively adjusting your model, do a _final_ evaluation with the holdout test set
6. DON'T TOUCH THE MODEL!!!

#### Ways to limit/deal with high variance.

- Get more data. With enough training data, even with floppy weights it'll get it right.

- Yeah, but often not possible/easy to get enough data for this.

- Get rid of columns that exhibit a high degree of collinearity with other columns.

- Yeah, but did we throw out some useful information for prediction? 
- ICR and schooling not the same thing.
- How many of the collinear columns should we throw away? Which ones?

- Or we could come up with ways to directly limit the variance through the cost function itself.

Let's try this.

# Preventing Overfitting - Regularization
Again, complex models are very flexible in the patterns that they can model but this also means that they can easily find patterns that are simply statistical flukes of one particular dataset rather than patterns reflective of the underlying data-generating process.

When a model has large weights, the model is "too confident". This translates to a model with high variance which puts it in danger of overfitting!

![](images/punishing_model_metaphor.jpeg)

We need to punish large (confident) weights by contributing them to the error function

**Some Types of Regularization:**

1. Reducing the number of features
2. Increasing the amount of data
3. Popular techniques: Ridge, Lasso, Elastic Net

#### Regularization for Linear Regression

Modify our squared error loss function.

$$ L = |\textbf{y} - X \textbf{w}|^2 + \lambda |\textbf{w}|^2 $$

with $|\textbf{w}|^2 = w_1^2 + w_2^2 + ... + w_m^2$ as sum of squares of the feature weights.

<img src = "Images/ridge_regression_geometric.png" width = 450>

Tug of war between:

Ridge cost: $ \lambda |\textbf{w}|^2 = \lambda (w_1^2 + w_2^2)  $
- L2 Regularization (Euclidean distance)

Least squares cost: $ |\textbf{y} - X\textbf{w}|^2 $
    

<img src = "Images/ridge_regression_geometric.png" width = 450>

Penalizes large weights: 
- **strongly** discourages large fluctuations in $\textbf{w}$ depending on training set.
- i.e. reduces Var[$\textbf{w}$]
- **Can lead to large performance boost on unseen data.**

- the tradeoff: repeating fitting on large number of distinct training sets (samples of population:
    - Average of $\textbf{w}$ is a little off from best fit to population

An oft-used picture:

<img src = "Images/biasvar_bullseye.png" width = 400/>

This really refers to the weights!
- Bullseye is $\textbf{w}$ fit on entire (unknown/unseen) population.
- Each dot is $\textbf{w}$ trained on a different sample (training set).

An oft-used picture:

<img src = "Images/biasvar_bullseye.png" width = 400/>

Idea of Ridge: 
- Tune $\lambda$ just right. This is something we input as external parameter to model. **Hyperparameter** 
- Cluster of $\textbf{w}$ little off the center of the bullseye
- But: tightly clustered.

OK let's do a Ridge regression.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

# train test split
from sklearn.model_selection import train_test_split

# preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# model validation: testing model variance with cross validation

from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
birds = sns.load_dataset('penguins')
birds = birds.dropna()

In [ ]:
birds.head()

Going to use the other features to predict the body mass of a penguin

In [ ]:
X = birds.drop('body_mass_g', axis=1)
y = birds['body_mass_g']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X ,y, random_state = 42)

Let's one-hot encode the nominal categoricals

In [ ]:
# Taking in other features (category)
ohe = OneHotEncoder(drop='first')
dummies = ohe.fit_transform(X_train[['species', 'island', 'sex']])

# Getting a DF
X_train_onehot = pd.DataFrame(dummies.todense(), columns=ohe.get_feature_names(), index=X_train.index)

X_train_onehot.head()

Attach this to dataframe with numerical variables

In [ ]:
X_train_numeric = X_train[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
X_train_df = pd.concat([X_train_numeric, X_train_onehot], axis=1)
X_train_df.head()

We have our training feature matrix:
- Apply transformation fit_transformed on train set to test feature matrix.

In [ ]:
test_dummies = ohe.transform(X_test[['species', 'island', 'sex']])
test_df = pd.DataFrame(test_dummies.todense(), columns=ohe.get_feature_names(),
                       index=X_test.index)
X_test_df = pd.concat([X_test[['bill_length_mm', 'bill_depth_mm',
                              'flipper_length_mm']], test_df], axis=1)
X_test_df.head()

In [ ]:
lr1 = LinearRegression()
lr1.fit(X_train_df, y_train)

In [ ]:
lr1.score(X_train_df, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = lr1.predict(X_test_df)
np.sqrt(mean_squared_error(y_pred, y_test))

Wow that's a good $R^2$ value!

- Estimate how we are doing on unseen data with cross validation.

In [ ]:
cv_results = cross_validate(X=X_train_df, y=y_train, estimator=lr1, cv=10, scoring=('r2', 'neg_mean_squared_error'),
                return_train_score=True
)

How well model explains training fold data in each iteration cross validation:

In [ ]:
train_res = cv_results['train_r2']
train_res

It's fitting well each time in the 10 iterations.

How does the validation look?

In [ ]:
test_res = cv_results['test_r2']
test_res

In [ ]:
test_res.mean()

In [ ]:
test_res.std(ddof = 1)

Good average test performance and relatively low variance.

In [ ]:
RMSE_train = np.sqrt(np.abs(cv_results['train_neg_mean_squared_error']))
RMSE_train

In [ ]:
RMSE_train.mean()

In [ ]:
RMSE_train.std(ddof = 1)

In [ ]:
RMSE_test = np.sqrt(np.abs(cv_results['test_neg_mean_squared_error']))
RMSE_test

Clearly larger average RMSE and variance of RMSE in the test set. But still pretty decent.

In [ ]:
RMSE_test.mean()

In [ ]:
RMSE_test.std(ddof = 1)

But...I'm a greedy man.

I want to do better than this. 
- I'm going to add some polynomials to get a more complex model.

My motivation:
    
- More complex model = better able to capture more complex relationships between mass and other variables.
- Better prediction.

#### Adding model complexity

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
pf = PolynomialFeatures(degree=3)
X_poly_train = pd.DataFrame(pf.fit_transform(X_train_df))

#y_train.to_csv('X_penguin.csv')

In [ ]:
X_poly_test = pf.transform(X_test_df)

In [ ]:
X_train_df.shape

In [ ]:
X_poly_train.shape

In [ ]:
poly_lr = LinearRegression()
poly_lr.fit(X_poly_train, y_train)

In [ ]:
poly_lr.score(X_poly_train, y_train)

In [ ]:
lr1.score(X_train_df, y_train)

About a 3% improvement: 
- that could mean money in other contexts.

In [ ]:
poly_cv_results = cross_validate(
                X=X_poly_train, 
                y=y_train,
                estimator=poly_lr, 
                cv=10,
                scoring=('r2', 'neg_mean_squared_error'),
                return_train_score=True
)

In [ ]:
poly_train_res = poly_cv_results['train_r2']
poly_train_res

In [ ]:
poly_train_res.mean()

In [ ]:
poly_train_res.std(ddof =1)

Wow...I'm ready to brag to my boss.

- But let's check the performance on the validation folds.

In [ ]:
poly_valid_res = poly_cv_results['test_r2']
poly_valid_res

In [ ]:
poly_valid_res.mean()

In [ ]:
poly_valid_res.std(ddof = 1)

In [ ]:
RMSE_polytest = np.sqrt(np.abs(poly_cv_results['test_neg_mean_squared_error']))
RMSE_polytest

In [ ]:
RMSE_polytest.mean()

In [ ]:
RMSE_polytest.std(ddof=1)

<center><img src = "Images/punch_chuck_norris.gif" width = 400/></center>
<center>You just got punched in the face by the bias-variance problem.</center>

Fit an overly complex model:
- Doesn't generalize well.

Let's try regularizing polynomial model:
$$  L = ||\textbf{y} - X \textbf{w}||_2^2 + \lambda |\textbf{w}|_2^2 $$


- reduce the floppiness/complexity of model
- but still keep *some* of the complexity added by these polynomial features.

- Reduce Var[$\textbf{w}$].
- Get model predictions more representative of population.

In [ ]:
ss = StandardScaler()
pf = PolynomialFeatures(degree=3)

# You should always be sure to _standardize_ your data before
# applying regularization!

X_train_processed = pf.fit_transform(ss.fit_transform(X_train_df))
X_test_processed = pf.transform(ss.transform(X_test_df))

Absolutely need to standardize/normalize features:

 $$ L = ||\textbf{y} - X \textbf{w}||_2^2 + \lambda |\textbf{w}|_2^2 $$

L2 regularization cost function makes no sense otherwise. 
- Weights will be on different scales if features not normalized.

In [ ]:
# 'Lambda' is the standard variable for the strength of the
# regularization (as in the above formulas), but since lambda
# is a key word in Python, these sklearn regularization tools
# use 'alpha' instead.

rr = Ridge(alpha=100, random_state=42)

rr.fit(X_train_processed, y_train)

In [ ]:
rr.score(X_train_processed, y_train)

In [ ]:
lr1.score(X_train_df, y_train)

Let' s cross validate this.

In [ ]:
rr_cv_results = cross_validate(
                X=X_train_processed, 
                y=y_train,
                estimator=rr, 
                cv=10,
                scoring=('r2', 'neg_mean_squared_error'),
                return_train_score=True
)

Get $R^2$ on train folds of cross validation trials:

In [ ]:
rr_cv_results['train_r2']

Get $R^2$ on validation folds of cross validation trials:

In [ ]:
rr_cv_results['test_r2']

In [ ]:
rr_cv_results['test_r2'].mean()

In [ ]:
rr_cv_results['test_r2'].std(ddof = 1)

In [ ]:
RMSE_rrtest = np.sqrt(np.abs(rr_cv_results['test_neg_mean_squared_error']))
RMSE_rrtest

In [ ]:
RMSE_rrtest.mean()

In [ ]:
RMSE_rrtest.std(ddof = 1)

L2 regularized polynomial model:
- A little bit worse than my basic linear model.
- Much much better than the un-regularized polynomial model.

Could I do better? Possibly.

- Tune hyperparameter $\lambda$.

#### Tuning hyperparameters

- Don't know what $\lambda$ will allow model to perform best on validation sets.
- Need to tune this.

- Adjust model stiffness/regularization parameter
- Assess model performance in validation testing.

### Optimizing the Regularization Hyperparameter

#### The most basic hyperparameter tuning method: Make a loop!

The regularization strength could sensibly be any nonnegative number, so there's no way to check "all possible" values. It's often useful to try several values that are different orders of magnitude.

In [ ]:
alphas = [1e-3, 1e-2, 1e-1, 1, 10, 100, 1e3, 1e4]
cv_scores = []

for alpha in alphas:
    rr = Ridge(alpha=alpha, random_state=42)
    cv_loop_results = cross_validate(
                X=X_train_processed, 
                y=y_train,
                estimator=rr, 
                cv=10,
                scoring=('neg_mean_squared_error'))
    cv_scores.append(np.mean(np.sqrt(np.abs(cv_loop_results['test_score']))))

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(x = np.log10(alphas), y = cv_scores, marker = 's', ax = ax)
ax.set_xlabel('Log(lambda)')
ax.set_ylabel('Mean RMSE')
ax.set_title('RMSE averaged on validation folds')
plt.show()

Could fine tune:
- But of hyperparameter values tried $\lambda = 100$ is best.

Now we *finally* report results on the true test set:
- We have not fit optimized on it.
- Have not tuned hyperparameters to see how well it performs on validation folds.

**Test/hold-out set is our true final gold standard**

In [ ]:
rr = Ridge(alpha = 100, random_state = 42)
rr.fit(X_train_processed, y_train)

In [ ]:
y_pred = rr.predict(X_test_processed)

In [ ]:
from sklearn.metrics import mean_squared_error
ridge_RMSE_holdout = np.sqrt(mean_squared_error(y_pred, y_test))
ridge_RMSE_holdout

Given the scale of the penguin body mass (g): this is good.

In [ ]:
sns.histplot(y);

#### Takeaways of what we just did

- Polynomial model: poor prediction performance.
- L2 regularized the polynomial regression model (Ridge regression) + tuning
- **Much** better test performance than unregularized polynomial model.

But:
    
- Our simple linear model with no polynomial worked well.
- Almost as well as polynomial features
- Think carefully before adding model complexity.

A simple model with less number of good quality predictive features may work as well if not better.

Logical extension:
- Features that are correlated but don't want to throw them away.
- L2 regularized linear model + tuning:
    - don't throw out features.
    - get better test performance than OLS by reducing weight variance.


Sometimes though: throwing away features might work better:
- Learn good features only with high predictive power
- Chuck the rest.

#### L1 Regularization (LASSO)

$$ L = ||\textbf{y} - X \textbf{w}||_2^2 + \lambda ||\textbf{w}||_1 $$

with $||\textbf{w}||_1 = |w_1| + |w_2| + ... + |w_m|$ as sum of absolute magnitude of the feature weights.

Taxi cab vs Euclidean distance:

<img src = "Images/metrics.png" width = 450/>

Circle in terms of L2 vs L1 distance:

"Circle" in L1:
<img src = "Images/taxcabgeometry.jpg" width = 400/>

"Circles" for different metrics

<img src = "Images/circles.png" width = 400/>

Why use the L1 magnitude $||\textbf{w}||_1$ for regularization?

LASSO encourages model weight sparsity: 
- prefers to drive weights $w_i$ for features with little predictive power to 0.

<img src = "Images/different_metric_regularization.png" width = 600 />

Perform LASSO regression with scikit-learn:

In [ ]:
from sklearn.linear_model import Lasso

Find the best LASSO model: tune regularization hyperparameter.

In [ ]:
alphas = [1, 10, 100, 1e3, 1e4]
cv_lasso_scores = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha, random_state=42, max_iter = 10000)
    cv_loop_results = cross_validate(
                X=X_train_processed, 
                y=y_train,
                estimator=lasso, 
                cv=10,
                scoring=('neg_mean_squared_error'))
    cv_lasso_scores.append(np.mean(np.sqrt(np.abs(cv_loop_results['test_score']))))

In [ ]:
alphas

In [ ]:
cv_lasso_scores

The best LASSO model:

In [ ]:
lasso_opt = Lasso(alpha=10, random_state=42,  max_iter = 100000)
lasso_opt.fit(X_train_processed, y_train)

y_pred = lasso_opt.predict(X_test_processed) # get final test prediction

In [ ]:
lasso_RMSE = np.sqrt(mean_squared_error(y_pred, y_test))
lasso_RMSE

In [ ]:
ridge_RMSE_holdout

Comparable between ridge and LASSO. LASSO tends to have higher weight variance than ridge.

But what's the real difference?

In [ ]:
print(rr.coef_)

In [ ]:
lasso_opt.coef_

## The Strategy Behind Ridge / Lasso / Elastic Net

Overfit models overestimate the relevance that predictors have for a target. Thus overfit models tend to have **overly large coefficients**. 

Generally, overfitting models come from a result of high model variance. High model variance can be caused by:

- having irrelevant or too many predictors
- multicollinearity
- large coefficients

#### Ridge 
When we introduce many features that:
- we believe may all have some predictive power.
- want to heavily penalize weight variance

#### LASSO

We have dataset with many highly correlated features:
- believe many are not actually adding to predictive power.
- willing to cut away marginally unimportant features.


Which is better:
- depends on dataset
- modeling goal 

### LEVEL UP - Elastic Net!
Naturally, the Elastic Net has the same interface through sklearn as the other regularization tools! The only difference is that we now have to specify how much of each regularization term we want. The name of the parameter for this (represented by $\rho$ above) in sklearn is `l1_ratio`.

In [ ]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet(alpha=10, l1_ratio=0.1, random_state=42)

enet.fit(X_train_processed, y_train)

In [ ]:
enet.score(X_train_processed, y_train)

In [ ]:
enet.score(X_test_processed, y_test)

Setting the `l1_ratio` to 1 is equivalent to the lasso:

In [ ]:
ratios = np.linspace(0.01, 1, 100)

In [ ]:
preds = []
for ratio in ratios:
    enet = ElasticNet(alpha=100, l1_ratio=ratio, random_state=42)
    enet.fit(X_train_processed, y_train)
    preds.append(enet.predict(X_test_processed[0].reshape(1, -1)))

In [ ]:
fig, ax = plt.subplots()

lasso = Lasso(alpha=100, random_state=42)
lasso.fit(X_train_processed, y_train)
lasso_pred = lasso.predict(X_test_processed[0].reshape(1, -1))

ax.plot(ratios, preds, label='elastic net')
ax.scatter(1, lasso_pred, c='k', s=70, label='lasso')
plt.legend();

#### Note on `ElasticNet()`
Is an Elastic Net with `l1_ratio` set to 0 equivalent to the ridge? In theory yes. But in practice no. It looks like the `ElasticNet()` predictions on the first test data point as `l1_ratio` shrinks are tending toward some value around 3400. Let's check to see what prediction `Ridge()` gives us:

In [ ]:
ridge = Ridge(alpha=10, random_state=42)
ridge.fit(X_train_processed, y_train)
ridge.predict(X_test_processed[0].reshape(1, -1))[0]

If you check the docstring for the `ElasticNet()` class you will see:
- that the function being minimized is slightly different from what we saw above; and
- that the results are unreliable when `l1_ratio` $\leq 0.01$.

**Exercise**: Visualize the difference in this case between `ElasticNet(l1_ratio=0.01)` and `Ridge()` by making a scatterplot of each model's predicted values for the first ten points in `X_test_processed`. Use `alpha=10` for each model.

        Level Up: Make a second scatterplot that compares the predictions on the same data
        points between ElasticNet(l1_ratio=1) and Lasso().

<details>
    <summary> Answer
    </summary>
    <code>fig, ax = plt.subplots()
enet_r = ElasticNet(alpha=10, l1_ratio=0.01, random_state=42)
enet_r.fit(X_train_processed, y_train)
preds_enr = enet_r.predict(X_test_processed[:10])
preds_ridge = ridge.predict(X_test_processed[:10])
ax.scatter(np.arange(10), preds_enr)
ax.scatter(np.arange(10), preds_ridge);</code>  
        </details>

<details>
    <summary>
        Level Up Answer
    </summary>
<code>fig, ax = plt.subplots()
enet_l = ElasticNet(alpha=10, l1_ratio=1, random_state=42)
enet_l.fit(X_train_processed, y_train)
preds_enl = enet_l.predict(X_test_processed[:10])
preds_lasso = lasso.predict(X_test_processed[:10])
ax.scatter(np.arange(10), preds_enl)
ax.scatter(np.arange(10), preds_lasso);</code>
    </details

#### Fitting Regularized Models with Cross-Validation
Our friend `sklearn` also includes tools that fit regularized regressions *with cross-validation*: `LassoCV`, `RidgeCV`, and `ElasticNetCV`.


**Exercise**: Use `RidgeCV` to fit a seven-fold cross-validated ridge regression model to our `X_train_processed` data and then calculate $R^2$ and the RMSE (root-mean-squared error) on our test set.


<details>
    <summary>
        Answer
    </summary>
    <code>rcv = RidgeCV(cv=7)
rcv.fit(X_train_processed, y_train)
rcv.score(X_test_processed, y_test)
np.sqrt(mean_squared_error(y_test, rcv.predict(X_test_processed)))</code>
    </details>